# Yorum Sıralama (Sorting Reviews)

* Bu bölüme kadar ürünleri sıralama işlemiyle ilgilendik. Bu sıralama işlemi yapılırken;
1. İş bilgisi açısından önemli olabilecek faktörler göz önünde bulundurulmalı
2. Eğer birden fazla faktör varsa bu faktörlerin etkileri standartlaştırılmalı, daha sonra ağırlıkla ifade edilmeli
3. Listeratürde bulduğumuz işlemleri iş bilgisi ile harmanlayıp kullanılmalı. 
* Bu bölümde ise yorumları sıralama işlemi yapacağız. 
* Yorumlar sıralanırken yorumların düşük veya yüksek puanlı olması sırasını belirlemez. 
* Peki bunu sıralarken nelere bakılmalıdır? Örneğin, 
1. Diğer kullanıcıların ilgili yorumları faydalı bulma ya da bulmaması durumuna göre sıralanabilir. 
2. Tarihe göre sıralama 
3. Uzer quality score 
Dolayısıyla tek bir faktör yorum sıralamayı etkilemez, ilgili iş problemine yönelik oluşturulabilecek farklı skorlar geliştirilir. 


In [31]:
import pandas as pd
import math
import scipy.stats as st

### Up-Down Diff Score = (up ratings) − (down ratings) 

Bir örnekle inceleyecek olursak;
1. 600 tane beğeni almış (Up) ve 400 tane dislike(down) almış. Total= 1000 
2. 5500 tane up 4500 tane down almış. Total= 10000

Yukardakilerden hangisini tercih ederdiniz?

up-downdan review 1 200, review 2 ise 1000 olarak çıkıyor. Ancak yüzdelere bakınca; review 1 % 60, review 2 ise %55tir. Dolayısıyla up down diffirence score yöntemiyle bu işlemi yapmak yeterince doğru olmaz. 

In [32]:
def score_up_down_diff(up, down):
    return up - down
score_up_down_diff(600,400)
score_up_down_diff(5500,4500)

1000

### Average rating = (up ratings) / (all ratings)

In [33]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)
score_average_rating(600,400)
score_average_rating(5500,4500)

0.55

- Review 1 için average rating 0.6 ve review 2 için average rating 0.55tir. 
- Gözlemleneceği üzere bu iki yöntem üzerinden average rating daha mantıklı görünmektedir. 
- Aslında bu score faydalı olma oranını bulur. 
- Bu yönteme biraz daha detaylı bakalım;
1. Review: 2 up 0 down total 2 
2. Review: 100 up 1 down total 101 
* Bu yönteme göre hesaplayınca 1. review 1 tam puan alırken 2. review 0.99 puan alır. Eğer bu işleme göre bir sıralama yapılırsa review1 daha yukarda bir yer alır. Bu da doğru bir sıralama olmaz. 
* Oran bilgisi çözüldü ama frekans bilgisini çözemedi. Öyle bir scorelama yapmalıyız ki hem oran hem frekans bilgisini çözmeli.


### Wilson Lower Bound Score (WLB Score)

* Bu yöntem bize ikili ikili etkileşimler (like-dislike vb.) sonucunda ortaya çıkan ölçme problemlerinde kullanılan bir  yöntemidir.


In [34]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [35]:
wilson_lower_bound(600, 400)

0.5693094295142663

In [36]:
wilson_lower_bound(5500, 4500)

0.5402319557715324

In [37]:
#bu yönteme göre net bir şekilde review 1 in daha iyi olduğu görülmüştür. 

In [38]:
wilson_lower_bound(2, 0)

0.3423802275066531

In [39]:
wilson_lower_bound(100, 1)

0.9460328420055449

### Case Study

In [40]:
#aşağıdaki bir veriseti var, ve bunlar yorumların faydalı bulunma durumunun scorelarıdır. 
up = [15, 70, 14, 4, 2, 5, 8, 37, 21, 52, 28, 147, 61, 30, 23, 40, 37, 61, 54, 18, 12, 68]
down = [0, 2, 2, 2, 15, 2, 6, 5, 23, 8, 12, 2, 1, 1, 5, 1, 2, 6, 2, 0, 2, 2]
comments = pd.DataFrame({"up": up, "down": down})

comments


,up,down
0,15,0
1,70,2
2,14,2
3,4,2
4,2,15
5,5,2
6,8,6
7,37,5
8,21,23
9,52,8


In [41]:
# score_pos_neg_diff
comments["score_pos_neg_diff"] = comments.apply(lambda x: score_up_down_diff(x["up"],
                                                                             x["down"]), axis=1)

# score_average_rating
comments["score_average_rating"] = comments.apply(lambda x: score_average_rating(x["up"], x["down"]), axis=1)

# wilson_lower_bound
comments["wilson_lower_bound"] = comments.apply(lambda x: wilson_lower_bound(x["up"], x["down"]), axis=1)

In [42]:
comments

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,15,0,15,1.000000,0.796117
1,70,2,68,0.972222,0.904258
2,14,2,12,0.875000,0.639772
3,4,2,2,0.666667,0.299993
4,2,15,-13,0.117647,0.032880
5,5,2,3,0.714286,0.358934
6,8,6,2,0.571429,0.325906
7,37,5,32,0.880952,0.749996
8,21,23,-2,0.477273,0.337550
9,52,8,44,0.866667,0.758348


In [43]:
comments.sort_values("wilson_lower_bound", ascending=False)

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
11,147,2,145,0.986577,0.952384
12,61,1,60,0.983871,0.914133
1,70,2,68,0.972222,0.904258
21,68,2,66,0.971429,0.901677
18,54,2,52,0.964286,0.878812
15,40,1,39,0.975610,0.874049
13,30,1,29,0.967742,0.838059
16,37,2,35,0.948718,0.831144
19,18,0,18,1.000000,0.824121
17,61,6,55,0.910448,0.818072
